# Import Library

In [2]:
import os
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
print(tf.config.list_physical_devices('GPU'))
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB4

[]


#### Set Seed 

In [2]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

#### Config 

In [3]:
IMAGE_PATH = f'./Crawling/Data/'

batch_size = 16

classes = 17

learning_rate = 1e-3

EPOCHS = 100

# Load Data 

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
)

val_datagen = ImageDataGenerator(
    rescale=1./255.,
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    IMAGE_PATH,
    target_size=[256,256],
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    IMAGE_PATH,
    target_size=[256,256],
    batch_size=batch_size,
    class_mode='categorical'
)

Found 5100 images belonging to 17 classes.
Found 5100 images belonging to 17 classes.


# Build Model 

In [6]:
with tf.device('/device:GPU:0'):
    model = EfficientNetB4(
        include_top=False,
        input_shape=(256,256,3),
        classes=classes)

71688192/71686520 [==============================] - 7s 0us/step


In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(17)

In [8]:
model = tf.keras.Sequential([
    model,
    global_average_layer,
    prediction_layer
])

#### Compile Model 

In [9]:
model.compile(
    optimizer = tf.keras.optimizers.SGD(learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb4 (Functional)  (None, 8, 8, 1792)        17673823  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1792)              0         
_________________________________________________________________
dense (Dense)                (None, 17)                30481     
Total params: 17,704,304
Trainable params: 17,579,097
Non-trainable params: 125,207
_________________________________________________________________


# Training 

In [ ]:
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

Epoch 1/100
 18/319 [>.............................] - ETA: 14:46 - loss: 7.2165 - accuracy: 0.0590

In [23]:
r = next(iter(train_generator))

In [40]:
img, label = r[0][0], r[1][0]

In [1]:
import tensorflow as tf

[]